In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Intersting ways to organize the DF when updating column (like converting Data)")
    .getOrCreate()
)

23/05/13 11:37:10 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/05/13 11:37:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/13 11:37:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/13 11:37:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
csvFile = ("../LearningSpark/data/7-departuredelays.csv")
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
df = (spark.read
    .schema(schema)
    .format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(csvFile))

In [4]:
df.show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [5]:
from pyspark.sql.functions import date_format, from_unixtime, unix_timestamp, to_timestamp

In [6]:
df_one = (
    df.select(
        date_format(
            from_unixtime(
            unix_timestamp('date', 'MMddHHmm')), 'MM-dd HH:mm').alias('date'), 
    *df.columns).drop(df.date)
)
df_one.show(3)


+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
+-----------+-----+--------+------+-----------+
only showing top 3 rows



In [ ]:
df_one_alternative = (
    df.select(
        date_format(
            to_timestamp('date', 'MMddHHmm'), 'MM-dd HH:mm').alias('date'),
    *df.columns).drop(df.date)
    )
)
df_one_alternative.show(3)
df_one_alternative.printSchema()

In [7]:
df_two = (
    df.select(
        date_format(
            from_unixtime(
            unix_timestamp('date', 'MMddHHmm')), 'MM-dd HH:mm').alias('date'), 
        *[col for col in df.columns if col != 'date']
    )
)

df_two.show(3)


+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
+-----------+-----+--------+------+-----------+
only showing top 3 rows



In [8]:
df_three = (
    df.withColumn("new_date",
        date_format( from_unixtime(
        unix_timestamp("date", 'MMddHHmm')), 'MM-dd HH:mm'))
    .drop('date')
    .withColumnRenamed("new_date", "date")
    .select(*df.columns)
)

df_three.show(3)


+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
+-----------+-----+--------+------+-----------+
only showing top 3 rows



In [9]:
df_four = (
    df.select(
        date_format( 
            from_unixtime(
            unix_timestamp("date", 'MMddHHmm')), 'MM-dd HH:mm').alias("date"),
        "delay", "distance", "origin", "destination"
        )
)

df_four.show(3)


+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
+-----------+-----+--------+------+-----------+
only showing top 3 rows



In [10]:
df_five = (
    df.withColumn("new_date",
        date_format( from_unixtime(
        unix_timestamp("date", 'MMddHHmm')), 'MM-dd HH:mm'))
    .drop('date')
    .withColumnRenamed("new_date", "date")
    .select("date", "delay", "distance", "origin", "destination")
)

df_five.show(3)


+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
+-----------+-----+--------+------+-----------+
only showing top 3 rows



In [11]:
df

DataFrame[date: string, delay: int, distance: int, origin: string, destination: string]